This is my 2nd competition on kaggle and as a beginner in data science I am trying Random Forest regressor for the current TPS-AUG competiton. I have structured the notebook based on the results, it basically depicts my understanding of the results and the subsequent step. As a beginner, I think this approach will help me and other beginners like me in future. Also It's a work in progress...

* P.S - I am all ears to Constructive criticism and suggestions to make my inferencing, data handling, and model generation better. 

# Importing

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-aug-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-aug-2021/test.csv")
submission = pd.read_csv("../input/tabular-playground-series-aug-2021/sample_submission.csv")

# I'll remove the ID column of both train and test, cause they are of no use here
train.drop(columns = 'id', inplace = True)
test.drop(columns = 'id', inplace =True)

# Check
print('train columns', train.columns)
print('test columns', test.columns)

# Extracting Information


In [ ]:
print('train shape', train.shape)
train.describe()

Huge data set, thus a simple describe is not going to captute the full picture, so I'll rather do a Box plot, and see how the data looks. 
* P.S - Any other better method for suggestion

In [ ]:
box = train.boxplot(figsize = (30,8), rot = 90 )


OOPs seems like box plot also gave a pretty weird outcome, any suggestion is welcome
* Conclusion that can be drawn is that,   
* feature f60 has pretty different scale compared to the rest.   
* I need to perform some scaling to make the visualisation sensible, but for that i need to know the presence of outliers in the data. 
* Something for later but, i might want to see how data model is affected w and w/o any scaling

## Need to check for outliers

In [ ]:
# converting the describe table into a spread dataframe 
spread = pd.DataFrame(train.describe().T)

# Calculating the IQR for each feature column
IQR = spread['75%'] - spread['25%']
spread['outliers'] = ( (spread['max'] > (spread['75%'] + 1.5 * IQR)) 
                         | (spread['min'] < (spread['25%'] - 1.5 * IQR)) )

# Outliers count 
spread['outliers_count'] = ((train < (spread['25%'] - 1.5 * IQR)) 
                              | (train > (spread['75%'] + 1.5 * IQR))).sum()

spread.head()

In [ ]:
# Visualising the outliers
plt.figure(figsize=(5,3))
ax = sns.countplot( x="outliers", data=spread, palette="Set2")
ax.set_title("Feature with Outliers")

Well there are multiple features that have outliers, so may be i'll have to tackle the outliers, maybe with min-max scaler or robust scaler. 
* P.s - Again it can change based on the dataset numbers and models that i would like to try, say a standard scalar or min-max scalar

In [ ]:
# Let's check for Null values 
print('Total Null values:', train.isnull().sum().sum())

**luckily no null values**

# Visualisation With Graphs

In [ ]:
fig, axes = plt.subplots(10,10,figsize=(15,12))
axes = axes.flatten()

for idx, ax in enumerate(axes):
    sns.kdeplot(data=train, x=f'f{idx}', 
                fill=True, 
                ax=ax)
    # sns.kdeplot(data=test, x=f'f{idx}', 
                # fill=True, 
                # ax=ax)
    ax.set_xticks([])
    # ax.set_yticks([])
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.spines['left'].set_visible(False)
    ax.set_title(f'f{idx}', loc='right', weight='bold', fontsize=10)

fig.supxlabel('Average by class (by feature)', ha='center', fontweight='bold')
fig.tight_layout()
plt.show()

### Inference
Thanks to the huge feature numbers, visualising them together is a daunting task. Anyways, couple of thing:
  * Aligning with the outliers calculation, the good number of data is out of normal, with hint of skewness
  * I plotted the data wiht their y axis, to get a better understanding of the scaling, and yeah, scaling has to be done. Well as mentioned previously I'll perform std scalar and robust scalar to see how they effect the model. 

## Correlation of Features

In [ ]:
# Correlation with pearson corr 

plt.figure(figsize=(20,20))
sns.heatmap(train.corr(), 
            annot=False, cmap="vlag",
            center = 0, linewidths=0.01,
            cbar_kws={"shrink": .5}, square = True, vmin=-0.1, vmax=0.1) 


### Inference
* Well the features are too less correlated, both positively and negatively with each other and the target, also the festures are lot less normally distributed, so I'll do a spearman corr heatmap, lets see if i see any changes.

In [ ]:
# Correlation with spearman corr
plt.figure(figsize=(20,20))
sns.heatmap(train.corr(method = 'spearman'), 
            annot=False, cmap="vlag",
            center = 0, linewidths=0.01,
            cbar_kws={"shrink": .5}, square = True, vmin=-0.1, vmax=0.1) 


### Inference: 
* Performed correaltion with spearman since a lot of features are not normal in nature but all in vain, still i think spearman will be better form, since the data is not normally distributed. 
* Both the heatmaps dont have much of difference,in terms of correlation between the variables as i was expecting better correlation when considering the data nature. 
* P.s So for feature selection in case if i have to do, i'll try using using mutual info or PCA, (data size is a factor)

# Model Creation

Obviously with such low correlations between the variables, an emsemble technique will come into the picture. Also since i am starting new, i'll do Random Forest regressor, one of the most used model algorithm. 
* P.s Also, i have tried multiple variation of random forest, but do to lack of a proper computing power, i'll show limited versions of the model runs. So if anyone has any sugestion as to how i can use such a huge data and run random forest, keep do comment and suggest.

## RF - baseline

In [ ]:
# Splitting
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train.iloc[: ,:-1], train.iloc[:,-1], 
                                                    test_size=0.4, random_state=2)
print("X-train shape", x_train.shape, "\nY-test shape" ,y_test.shape)

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor()
# Train the model on training data
rf.fit(x_train, y_train)
# Use the forest's predict method on the test data
y_pred = rf.predict(x_test)
# Calculate the absolute errors
errors = abs(y_pred - y_test)
# Print out the mean absolute error (mae)
print('Base line Mean Absolute Error', round(np.mean(errors), 4), 'degrees.')

### Inference
I am pretty sure this prediction can be improved with parametric tuning. 
* The following line of code is ideally derived using a CV + RF (multiple paramerters), but i couldn't do it, **ran out of RAM**. Possible reason for running out of RAM is too big of a data set and too many features to work with. 
* The model is under progress, i'll try redducing the dimensions to work around the data and then improve it as well. 
* If anyone has any suggestions, lets discuss

## RF - 2.0, 2.1
Well i wanted to get the proper parameters for the model, but the runtime was insane for each. Must be a rookie mistake. 
* So the work around is, i'll show the output of each the runs their degree of error and run the final one again to show the improvement

#### Rf -2.0
* parameters : n_estimators=200, max_depth = 10, 
                           min_samples_split = 25, min_samples_leaf = 5,
                           verbose = 1
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed: **47.1min finished**
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
 **Base line Mean Absolute Error 6.18 degrees.**
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    2.2s finished

#### Rf - 2.1
* Parameters: n_estimators=200, max_depth = 20, 
                           min_samples_split = 50, min_samples_leaf = 10,
                           verbose = 1

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed: **77.6min finished**
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
 **Base line Mean Absolute Error 6.18 degrees.**
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    4.5s finished

## RF - 2.2

In [ ]:
# Splitting
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train.iloc[: ,:-1], train.iloc[:,-1], 
                                                    test_size=0.4, random_state=2)
print("X-train shape", x_train.shape, "\nY-test shape" ,y_test.shape)

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators=500, max_depth = 20, 
                           min_samples_split = 50, min_samples_leaf = 10,
                           verbose = 1)

# Train the model on training data
rf.fit(x_train, y_train)

# Use the forest's predict method on the test data
y_pred = rf.predict(x_test)
# Calculate the absolute errors
errors = abs(y_pred - y_test)
# Print out the mean absolute error (mae)
print('Base line Mean Absolute Error', round(np.mean(errors), 4), 'degrees.')


One thing is clear, the time needed for the code is too much and not worth the decrease in error. Need better method for this dataset.

In [ ]:
submission['loss'] = rf.predict(test)
submission
submission.to_csv("submission_rf2.2.csv", index = False)

Work in progress....